In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("Iceberg + LocalStack")

    # Default demo catalog
    .config("spark.sql.catalog.demo", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.demo.type", "rest")
    .config("spark.sql.catalog.hse_catalog.uri", "http://host.docker.internal:8181")
    .config("spark.sql.catalog.demo.warehouse", "s3a://test/warehouse")
    .config("spark.sql.catalog.demo.io-impl", "org.apache.iceberg.hadoop.HadoopFileIO")
    .config("spark.hadoop.fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")


    # ✅ New catalog
    .config("spark.sql.catalog.hse_catalog", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.hse_catalog.type", "rest")
    .config("spark.sql.catalog.hse_catalog.uri", "http://iceberg-rest:8181")
    .config("spark.sql.catalog.hse_catalog.warehouse", "s3a://test/hse_data/")
    .config("spark.sql.catalog.hse_catalog.io-impl", "org.apache.iceberg.hadoop.HadoopFileIO")

    # LocalStack S3
    .config("spark.hadoop.fs.s3a.endpoint", "http://host.docker.internal:4566")
    .config("spark.hadoop.fs.s3a.access.key", "user")
    .config("spark.hadoop.fs.s3a.secret.key", "password")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")

    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/23 17:58:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.option("header", "true").csv("s3a://test/my_data/logs.csv")

25/04/23 17:58:28 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [3]:
df.show()

+-----+--------------------+--------------------+
|level|          event_time|             message|
+-----+--------------------+--------------------+
| INFO|2025-04-23T10:00:00Z|    Startup complete|
|ERROR|2025-04-23T10:05:00Z|NullPointerException|
| WARN|2025-04-23T10:10:00Z|      Disk space low|
+-----+--------------------+--------------------+



In [4]:
spark.sql("CREATE NAMESPACE IF NOT EXISTS hse_catalog.default")


DataFrame[]

In [8]:
from pyspark.sql.types import DoubleType, FloatType, LongType, StructType,StructField, StringType


df = spark.createDataFrame([], schema)


In [10]:
spark.sql("""
CREATE TABLE IF NOT EXISTS hse_catalog.default.events (
  id STRING,
  timestamp TIMESTAMP,
  severity STRING,
  message STRING
)
USING iceberg
TBLPROPERTIES (
  'format-version'='2'
)
""")


Py4JJavaError: An error occurred while calling o56.sql.
: org.apache.iceberg.exceptions.ServiceFailureException: Server error: RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.iceberg.rest.ErrorHandlers$DefaultErrorHandler.accept(ErrorHandlers.java:162)
	at org.apache.iceberg.rest.ErrorHandlers$TableErrorHandler.accept(ErrorHandlers.java:109)
	at org.apache.iceberg.rest.ErrorHandlers$TableErrorHandler.accept(ErrorHandlers.java:93)
	at org.apache.iceberg.rest.HTTPClient.throwFailure(HTTPClient.java:183)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:292)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:226)
	at org.apache.iceberg.rest.HTTPClient.post(HTTPClient.java:337)
	at org.apache.iceberg.rest.RESTClient.post(RESTClient.java:112)
	at org.apache.iceberg.rest.RESTSessionCatalog$Builder.create(RESTSessionCatalog.java:642)
	at org.apache.iceberg.CachingCatalog$CachingTableBuilder.lamb